<a href="https://colab.research.google.com/github/prikshit-2000/Tensorflow/blob/main/Custom_Training_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Dense , Flatten , Input
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as mticker

In [22]:
train_data = tfds.load('fashion_mnist',split = 'train')
test_data = tfds.load('fashion_mnist' , split = 'test')

In [23]:
class_names = ["T-shirt/top", "Trouser/pants", "Pullover shirt", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [24]:
def format_image(data):
  image = data['image']
  image  = tf.reshape(image,[-1])
  image = tf.cast(image,'float32')
  image = image/255.0

  return image,data['label']


In [25]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

In [26]:
batch_size = 64
train = train_data.shuffle(buffer_size = 1024).batch(batch_size)
test = test_data.batch(batch_size)

In [27]:
def base_model():
  inputs = Input(shape = (784,) , name = 'clothing')
  x = Dense(64 , activation = 'relu' , name = 'dense_1')(inputs)
  x = Dense(64 , activation = 'relu' , name = 'dense_2')(x)
  x  = Dense(10, activation = 'softmax' , name = 'predictions')(x)
  model = Model(inputs = inputs , outputs = x)
  return model


In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [29]:
train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [30]:
def apply_gradient(optimizer, model , x , y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = loss_object(y,logits)

  grads = tape.gradient(loss , model.trainable_weights)
  optimizer.apply_gradients(zip(grads,model.trainable_weights))

  return logits,loss

  

In [34]:
def train_data_for_one_epoch():
  losses = []

  for step,(batch_x,batch_y) in enumerate(train):
    logits,loss = apply_gradient(optimizer , model , batch_x , batch_y)
    losses.append(loss)

    train_metric(batch_y,logits)
  return losses

In [35]:
def perform_validation():
  losses = []

  for batch_x,batch_y in (test):
    logits = model(batch_x)
    val_loss = loss_object(batch_y,logits,)
    losses.append(val_loss)
    val_metric(batch_y,logits)
  return losses


In [36]:
model = base_model()

epochs = 20
val_loss , train_loss = [] , []
for i in range(epochs):
  losses_train = train_data_for_one_epoch()
  train_acc = train_metric.result()

  losses_val = perform_validation()
  val_acc = val_metric.result()


  losses_train_mean = np.mean(losses_train)
  losses_val_mean = np.mean(losses_val)

  train_loss.append(losses_train_mean)
  val_loss.append(losses_val_mean)
  print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (i, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))
  train_metric.reset_states()
  val_metric.reset_states()  


 Epoch 0: Train loss: 0.4842  Validation Loss: 0.4227, Train Accuracy: 0.8261, Validation Accuracy 0.8459

 Epoch 1: Train loss: 0.3668  Validation Loss: 0.3905, Train Accuracy: 0.8667, Validation Accuracy 0.8603

 Epoch 2: Train loss: 0.3342  Validation Loss: 0.3667, Train Accuracy: 0.8780, Validation Accuracy 0.8714

 Epoch 3: Train loss: 0.3156  Validation Loss: 0.3620, Train Accuracy: 0.8829, Validation Accuracy 0.8704

 Epoch 4: Train loss: 0.2990  Validation Loss: 0.3631, Train Accuracy: 0.8905, Validation Accuracy 0.8706

 Epoch 5: Train loss: 0.2870  Validation Loss: 0.3425, Train Accuracy: 0.8958, Validation Accuracy 0.8783

 Epoch 6: Train loss: 0.2784  Validation Loss: 0.3426, Train Accuracy: 0.8982, Validation Accuracy 0.8777

 Epoch 7: Train loss: 0.2695  Validation Loss: 0.3500, Train Accuracy: 0.9000, Validation Accuracy 0.8756

 Epoch 8: Train loss: 0.2614  Validation Loss: 0.3501, Train Accuracy: 0.9038, Validation Accuracy 0.8755

 Epoch 9: Train loss: 0.2552  Valida